In [1]:
import pickle
import os
import torch
import torch.nn.functional as F

In [2]:
results_path='/tsi/hi-paris/FCCLIP_results/All_results/INvocab/fcclip_cocopan_soft_label_calcul_loss_'
image_id='frankfurt_000000_003025'

In [3]:
def read_data(results_path, image_id, iter):
    path_cls=os.path.join(results_path, "Softlabels", "iter_"+str(iter), "mask_cls_"+image_id)
    path_pred=os.path.join(results_path, "Softlabels", "iter_"+str(iter), "mask_pred_"+image_id)
    path_cls_results=os.path.join(results_path, "Softlabels", "iter_"+str(iter), "cls_results_"+image_id) 
    with open(path_cls, 'rb') as file:
        mask_cls_results=pickle.load(file)
    with open(path_pred, 'rb') as file:
        mask_pred_results=pickle.load(file)
    with open(path_cls_results, 'rb') as file:
        cls_results=pickle.load(file)
    return mask_cls_results, mask_pred_results, cls_results

In [3]:
def read_data_jsons():
    path_json_data="/tsi/hi-paris/FCCLIP_results/All_results/INvocab/fcclip_cocopan_soft_label_calcul_loss_new/datalosses/iter_0/json_data_0"
    path_output_0="/tsi/hi-paris/FCCLIP_results/All_results/INvocab/fcclip_cocopan_soft_label_calcul_loss_new/datalosses/iter_0/outputs_0"
    path_targets_0="/tsi/hi-paris/FCCLIP_results/All_results/INvocab/fcclip_cocopan_soft_label_calcul_loss_new/datalosses/iter_0/targets_0"
    with open(path_json_data, 'rb') as file:
        json_data=pickle.load(file) 
    with open(path_output_0, 'rb') as file:
        outputs=pickle.load(file)
    with open(path_targets_0, 'rb') as file:
        targets=pickle.load(file)
    return json_data, outputs, targets

In [4]:
json_data, outputs, targets=read_data_jsons()

In [5]:
src_logits = outputs.float()
idx = json_data['idx']
target_classes_o = torch.cat([t["labels"][J] for t, (_, J) in zip(targets, json_data['indices'])])
target_classes = torch.full(
            src_logits.shape[:2], json_data['num_classes'], dtype=torch.int64, device=src_logits.device)
target_classes[idx] = target_classes_o

loss_ce = F.cross_entropy(src_logits.transpose(1, 2), target_classes, json_data['weight'])
losses = {"loss_ce": loss_ce}

In [11]:
print(json_data['weight'].shape, target_classes.shape)
json_data['weight']

torch.Size([20]) torch.Size([4, 250])


tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 0.1000])

In [12]:
target_classes

tensor([[19, 19, 19, 19, 19, 19, 19, 19, 11, 19, 19, 19, 19, 13, 13, 19, 19, 19,
         13, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 18, 19, 19, 19, 19, 19,
         13, 19, 19, 19,  5, 19, 19, 19, 19, 19, 19, 11, 19, 19, 19, 19, 19, 13,
         19, 18, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 13, 19, 19, 19, 13, 19,
         19, 19, 19, 19, 19, 13, 19, 19, 18, 13, 19, 19, 19, 19, 19, 10, 19, 19,
          8, 19, 19, 19, 19, 19, 13, 19, 19, 19, 19, 19, 19, 19, 19, 13, 19, 19,
         19, 19, 19, 13, 19, 19, 19,  7, 19, 19, 19, 19, 19, 19, 11, 19, 19, 19,
         19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 13, 19, 19, 19, 19, 19,
         19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 13, 11, 19, 19, 19, 19, 19,
         19, 19, 19, 19, 13, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19,
         19, 13, 13, 19, 19, 13, 13, 19, 19, 19, 19,  0, 19, 19, 19, 19, 19, 12,
         19, 19, 19, 11, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19,
         19, 19, 19, 19,  2,

In [57]:
loss_ce

tensor(0.4469, grad_fn=<NllLoss2DBackward0>)

In [26]:
F.cross_entropy(src_logits.transpose(1, 2)[0].T[0], target_classes[0][0], json_data['weight'])

tensor(0.0405, grad_fn=<NllLossBackward0>)

In [27]:
src_logits.transpose(1, 2)[0].T[0]

tensor([-0.3586, -0.4592,  2.8711,  2.5078,  0.3867,  1.1680,  6.5586,  2.6660,
         1.7539, -1.3252,  2.9688,  0.8325, -0.3364,  0.8530,  0.7373, -0.0676,
        -0.9175, -2.3281, -1.3447,  9.8594], grad_fn=<SelectBackward0>)

In [28]:
target_classes[0][0]

tensor(19)

In [48]:
log_softmax

tensor([-10.2585, -10.3591,  -7.0287,  -7.3920,  -9.5131,  -8.7319,  -3.3412,
         -7.2338,  -8.1459, -11.2250,  -6.9311,  -9.0673, -10.2363,  -9.0468,
         -9.1625,  -9.9674, -10.8173, -12.2280, -11.2446,  -0.0405],
       grad_fn=<LogSoftmaxBackward0>)

In [55]:
log_softmax.gather(-1, target.unsqueeze(-1))

tensor([-0.0405], grad_fn=<GatherBackward0>)

In [68]:
input=src_logits.transpose(1, 2)[0].T[0]
target=target_classes[0][0]
weight= json_data['weight']


# Apply log softmax to the input along the class dimension
log_softmax = F.log_softmax(input)

# Convert target to long type if not already
target = target.long()

# Gather the log softmax values corresponding to the target classes
log_probs = log_softmax.gather(-1, target.unsqueeze(-1)).squeeze(0)

/tmp/ipykernel_260685/153533964.py:7: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  log_softmax = F.log_softmax(input)


In [69]:
loss
log_probs

tensor(-0.0405, grad_fn=<SqueezeBackward1>)

In [73]:
log_softmax

tensor([-10.2585, -10.3591,  -7.0287,  -7.3920,  -9.5131,  -8.7319,  -3.3412,
         -7.2338,  -8.1459, -11.2250,  -6.9311,  -9.0673, -10.2363,  -9.0468,
         -9.1625,  -9.9674, -10.8173, -12.2280, -11.2446,  -0.0405],
       grad_fn=<LogSoftmaxBackward0>)

In [70]:


if weight is not None:
    # Apply class weights if provided
    log_probs = log_probs * weight[target]

# Compute the negative log likelihood loss
loss = -log_probs.mean()

In [71]:
log_probs

tensor(-0.0040, grad_fn=<MulBackward0>)

In [6]:
print("Inputs : ")
print("pred_logits : ",outputs)

Inputs : 
pred_logits :  tensor([[[-3.5864e-01, -4.5923e-01,  2.8711e+00,  ..., -2.3281e+00,
          -1.3447e+00,  9.8594e+00],
         [ 3.8945e+00,  8.5781e+00,  2.6831e-01,  ..., -3.5742e-01,
           1.2451e-02,  9.9375e+00],
         [ 1.0234e+00,  1.1924e+00,  4.3915e-02,  ...,  3.7891e+00,
           2.7188e+00,  1.1656e+01],
         ...,
         [ 4.7461e+00,  8.8750e+00,  8.9795e-01,  ..., -1.7432e+00,
          -1.8445e-01,  1.0820e+01],
         [-1.6914e+00, -1.7783e+00,  1.0352e+00,  ..., -1.8564e+00,
          -4.6973e-01,  1.0852e+01],
         [-2.0020e+00, -1.6787e+00,  3.1836e+00,  ..., -3.5117e+00,
          -1.3008e+00,  1.0461e+01]],

        [[-3.8110e-01, -3.4570e-01,  6.0898e+00,  ..., -2.6406e+00,
          -8.7549e-01,  1.0281e+01],
         [ 5.1289e+00,  1.2453e+01,  8.8818e-01,  ..., -2.7168e+00,
          -1.1309e+00,  7.3516e+00],
         [ 4.9072e-01,  1.6504e+00,  8.3691e-01,  ...,  2.3711e+00,
           1.7812e+00,  1.0109e+01],
         ...,


In [8]:
print("pred_logits shape : ",outputs.shape)

pred_logits shape :  torch.Size([4, 250, 20])


In [8]:
target_classes.shape

torch.Size([4, 250])

In [9]:
target_classes.shape,target_classes

(torch.Size([4, 250]),
 tensor([[19, 19, 19, 19, 19, 19, 19, 19, 11, 19, 19, 19, 19, 13, 13, 19, 19, 19,
          13, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 18, 19, 19, 19, 19, 19,
          13, 19, 19, 19,  5, 19, 19, 19, 19, 19, 19, 11, 19, 19, 19, 19, 19, 13,
          19, 18, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 13, 19, 19, 19, 13, 19,
          19, 19, 19, 19, 19, 13, 19, 19, 18, 13, 19, 19, 19, 19, 19, 10, 19, 19,
           8, 19, 19, 19, 19, 19, 13, 19, 19, 19, 19, 19, 19, 19, 19, 13, 19, 19,
          19, 19, 19, 13, 19, 19, 19,  7, 19, 19, 19, 19, 19, 19, 11, 19, 19, 19,
          19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 13, 19, 19, 19, 19, 19,
          19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 13, 11, 19, 19, 19, 19, 19,
          19, 19, 19, 19, 13, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19,
          19, 13, 13, 19, 19, 13, 13, 19, 19, 19, 19,  0, 19, 19, 19, 19, 19, 12,
          19, 19, 19, 11, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 1

In [10]:

#print("targets  : ",targets)


for i in  range(len(targets)):
    print("Element ", i)
    eleme=targets[i]
    print("len(eleme['labels']) :",len(eleme['labels']))
    print('len(eleme["masks"]) :',len(eleme["masks"]))
    print(eleme)
    break


Element  0
len(eleme['labels']) : 36
len(eleme["masks"]) : 36
{'labels': tensor([ 0,  1,  2,  5,  7,  8, 10, 11, 11, 11, 11, 11, 11, 12, 13, 13, 13, 13,
        13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 18, 18, 18]), 'masks': tensor([[[False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         ...,
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False]],

        [[False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         ...,
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False]],

        [[False,

In [16]:
out_float=outputs.float()

In [17]:
input_loss=out_float.transpose(1, 2)

In [18]:
input_loss.shape

torch.Size([4, 20, 250])

In [36]:
[sum(k) for k in outputs[0].T]

[tensor(273.5000, dtype=torch.float16, grad_fn=<AddBackward0>),
 tensor(402.5000, dtype=torch.float16, grad_fn=<AddBackward0>),
 tensor(377., dtype=torch.float16, grad_fn=<AddBackward0>),
 tensor(390.5000, dtype=torch.float16, grad_fn=<AddBackward0>),
 tensor(224.5000, dtype=torch.float16, grad_fn=<AddBackward0>),
 tensor(92.8750, dtype=torch.float16, grad_fn=<AddBackward0>),
 tensor(380.2500, dtype=torch.float16, grad_fn=<AddBackward0>),
 tensor(67.8125, dtype=torch.float16, grad_fn=<AddBackward0>),
 tensor(215.2500, dtype=torch.float16, grad_fn=<AddBackward0>),
 tensor(72.9375, dtype=torch.float16, grad_fn=<AddBackward0>),
 tensor(47.3125, dtype=torch.float16, grad_fn=<AddBackward0>),
 tensor(383., dtype=torch.float16, grad_fn=<AddBackward0>),
 tensor(239.8750, dtype=torch.float16, grad_fn=<AddBackward0>),
 tensor(1025., dtype=torch.float16, grad_fn=<AddBackward0>),
 tensor(562.5000, dtype=torch.float16, grad_fn=<AddBackward0>),
 tensor(304.5000, dtype=torch.float16, grad_fn=<AddBack

In [40]:
target_classes.shape, outputs.shape

(torch.Size([4, 250]), torch.Size([4, 250, 20]))

In [51]:
target_classes[0]

tensor([19, 19, 19, 19, 19, 19, 19, 19, 11, 19, 19, 19, 19, 13, 13, 19, 19, 19,
        13, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 18, 19, 19, 19, 19, 19,
        13, 19, 19, 19,  5, 19, 19, 19, 19, 19, 19, 11, 19, 19, 19, 19, 19, 13,
        19, 18, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 13, 19, 19, 19, 13, 19,
        19, 19, 19, 19, 19, 13, 19, 19, 18, 13, 19, 19, 19, 19, 19, 10, 19, 19,
         8, 19, 19, 19, 19, 19, 13, 19, 19, 19, 19, 19, 19, 19, 19, 13, 19, 19,
        19, 19, 19, 13, 19, 19, 19,  7, 19, 19, 19, 19, 19, 19, 11, 19, 19, 19,
        19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 13, 19, 19, 19, 19, 19,
        19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 13, 11, 19, 19, 19, 19, 19,
        19, 19, 19, 19, 13, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19,
        19, 13, 13, 19, 19, 13, 13, 19, 19, 19, 19,  0, 19, 19, 19, 19, 19, 12,
        19, 19, 19, 11, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19,
        19, 19, 19, 19,  2, 19, 19, 19, 

In [62]:
outputs[0].T[0]

tensor([-0.3586,  3.8945,  1.0234,  2.1309,  1.2266, -0.2834,  0.0754, -0.3267,
         0.6885,  1.9277,  2.0684,  0.5454,  3.1543,  0.2759,  2.0938, -0.6592,
         8.0781,  0.5771,  1.6709,  1.0430,  1.8721,  0.1394, -1.4746,  1.5469,
         6.2031,  1.6035,  4.2734,  0.1158, -0.3057,  1.4463,  1.7266,  1.1553,
         1.3643,  1.1328,  1.1387,  0.1415,  1.4766,  1.5195,  0.3533,  1.6240,
         0.7944,  1.8320,  1.1074, -0.5303,  0.3271,  1.8438,  2.0098, -0.7231,
         2.1641,  2.3984,  2.0332,  1.6699,  1.4258,  1.8115, -0.3203,  2.0918,
         1.3564,  2.9355,  1.6191,  3.2969,  0.2903,  1.5498,  1.9902, -0.8530,
         1.7578,  1.7451,  2.1406, -1.0986,  0.0757,  0.8169,  1.7695,  2.0547,
         1.9258,  1.1289,  2.4648,  0.4570, -0.0269,  1.6221,  1.1660,  0.8901,
         0.8076,  1.6533,  1.1787,  2.1055, -0.5259, -0.7295,  2.3301,  0.3372,
         1.1885, -0.2803,  0.8252,  1.6719,  0.0523, -0.5654, -0.1072,  1.8574,
         1.6689,  3.9590, -0.8008,  1.13

In [74]:
src_logits = outputs.float()
idx = json_data['idx']
target_classes_o = torch.cat([t["labels"][J] for t, (_, J) in zip(targets, json_data['indices'])])
target_classes = torch.full(
            src_logits.shape[:2], json_data['num_classes'], dtype=torch.int64, device=src_logits.device)
target_classes[idx] = target_classes_o

loss_ce = F.cross_entropy(src_logits.transpose(1, 2), src_logits.transpose(1, 2), json_data['weight'])
losses = {"loss_ce": loss_ce}

In [75]:
loss_ce

tensor(99.3829, grad_fn=<DivBackward1>)

In [80]:
input=src_logits.transpose(1, 2)[0].T[0]
target=src_logits.transpose(1, 2)[0].T[0]
weight= json_data['weight']


# Apply log softmax to the input along the class dimension
log_softmax = F.log_softmax(input)

# Convert target to long type if not already
target = target.long()

# Gather the log softmax values corresponding to the target classes
log_probs = log_softmax.gather(1, target.unsqueeze(1)).squeeze(1)

/tmp/ipykernel_260685/67097343.py:7: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  log_softmax = F.log_softmax(input)


IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [84]:
log_softmax.gather(0, target.unsqueeze(1))

RuntimeError: Index tensor must have the same number of dimensions as input tensor